In [7]:
import numpy as np                                          
from mindquantum.core.circuit import Circuit                
from mindquantum.core.gates import H, RX, RY, RZ,X    
from mindquantum.core.parameterresolver import PRGenerator  
from mindquantum.simulator import Simulator
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split   
from mindquantum.algorithm.library import amplitude_encoder
from mindquantum.algorithm.nisq import IQPEncoding
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
from mindquantum.core.operators import QubitOperator           # 导入QubitOperator模块，用于构造泡利算符
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
import mindspore as ms                                                                         # 导入mindspore库并简写为ms
from mindquantum.framework import MQLayer,MQN2Layer                                              # 导入MQLayer
# 导入HardwareEfficientAnsatz
from mindquantum.core.gates import RY           
import torch
from torchvision import datasets, transforms# 导入量子门RY
from scipy.ndimage import zoom
import random
from data_processing import PCA_data_preprocessing

In [2]:
# 下载和加载 MNIST 数据集
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
X_train, X_test, y_train, y_test = PCA_data_preprocessing(mnist_dataset)

In [24]:
IQPEncoder = IQPEncoding(n_feature=10)
encoder = IQPEncoder.circuit
encoder = encoder.no_grad()  
encoder.as_encoder()   
# encoder.summary()                     # 总结Encoder
# encoder.svg()

┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q0: ──┨ H ┠─┨ RZ(alpha0) ┠───■─────────────────────────────■─────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛   ┃                             ┃                    
      ┏━━━┓ ┏━━━━━━━━━━━━┓ ┏━┻━┓ ┏━━━━━━━━━━━━━━━━━━━━━┓ ┏━┻━┓                  
q1: ──┨ H ┠─┨ RZ(alpha1) ┠─┨╺╋╸┠─┨ RZ(alpha0 * alpha1) ┠─┨╺╋╸┠───■───[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛ ┗━━━┛ ┗━━━━━━━━━━━━━━━━━━━━━┛ ┗━━━┛   ┃              
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                     ┏━┻━┓            
q2: ──┨ H ┠─┨ RZ(alpha2) ┠─────────────────────────────────────┨╺╋╸┠─[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                     ┗━━━┛            
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q3: ──┨ H ┠─┨ RZ(alpha3) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q4: ──┨ H ┠─┨ RZ(alpha4) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q5: ──┨ H ┠─┨ RZ(alpha5) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q6: ──┨ H ┠─┨ RZ(alpha6) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q7: ──┨ H ┠─┨ RZ(alpha7) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q8: ──┨ H ┠─┨ RZ(alpha8) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
      ┏━━━┓ ┏━━━━━━━━━━━━┓                                                      
q9: ──┨ H ┠─┨ RZ(alpha9) ┠───────────────────────────────────────────[red bold]↯[/]─         
      ┗━━━┛ ┗━━━━━━━━━━━━┛                                                      
                                                                                
q0: ──────────────────────────────────────────────────────────────────────────[red bold]↯[/]─
                                                                                
                                                                                
q1: ────────────────────────────■─────────────────────────────────────────────[red bold]↯[/]─
                                ┃                                               
      ┏━━━━━━━━━━━━━━━━━━━━━┓ ┏━┻━┓                                             
q2: ──┨ RZ(alpha1 * alpha2) ┠─┨╺╋╸┠───■─────────────────────────────■─────────[red bold]↯[/]─
      ┗━━━━━━━━━━━━━━━━━━━━━┛ ┗━━━┛   ┃                             ┃           
                                    ┏━┻━┓ ┏━━━━━━━━━━━━━━━━━━━━━┓ ┏━┻━┓         
q3: ────────────────────────────────┨╺╋╸┠─┨ RZ(alpha2 * alpha3) ┠─┨╺╋╸┠───■───[red bold]↯[/]─
                                    ┗━━━┛ ┗━━━━━━━━━━━━━━━━━━━━━┛ ┗━━━┛   ┃     
                                                                        ┏━┻━┓   
q4: ────────────────────────────────────────────────────────────────────┨╺╋╸┠─[red bold]↯[/]─
                                                                        ┗━━━┛   
                                                                                
q5: ──────────────────────────────────────────────────────────────────────────[red bo

In [17]:
param_rd = []
IQPEncoder = IQPEncoding(n_feature=X_train[0].shape[0])
IQPEncoder.data_preparation(data=X_train[0])
pg = IQPEncoder.circuit.parameter_resolver()
for k, param in pg.items():
    print(k,param)
    param_rd.append(param)
param_rd = np.array(param_rd)

param_rd

alpha0 1.0
alpha0 * alpha1 1.0
alpha1 1.0
alpha1 * alpha2 1.0
alpha2 1.0
alpha2 * alpha3 1.0
alpha3 1.0
alpha3 * alpha4 1.0
alpha4 1.0
alpha4 * alpha5 1.0
alpha5 1.0
alpha5 * alpha6 1.0
alpha6 1.0
alpha6 * alpha7 1.0
alpha7 1.0
alpha7 * alpha8 1.0
alpha8 1.0
alpha8 * alpha9 1.0
alpha9 1.0


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [25]:
ansatz = HardwareEfficientAnsatz(6, single_rot_gate_seq=[RY], entangle_gate=X, depth=2).circuit     # 通过
hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
ansatz = ansatz.as_ansatz()
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(2)                                                     # 设置生成随机数的种子
circuit = encoder+ ansatz.as_ansatz()         
sim = Simulator('mqvector', 10)
grad_ops = sim.get_expectation_with_grad(hams,
                                         circuit,
                                         parallel_worker=5)
QuantumNet = MQLayer(grad_ops)          # 搭建量子神经网络

In [26]:
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集



loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')            # 通过SoftmaxCrossEntropyWithLogits定义损失函数，sparse=True表示指定标签使用稀疏格式，reduction='mean'表示损失函数的降维方法为求平均值
opti = Adam(QuantumNet.trainable_params(), learning_rate=0.02)                  # 通过Adam优化器优化Ansatz中的参数，需要优化的是Quantumnet中可训练的参数，学习率设为0.1

model = Model(QuantumNet, loss, opti, metrics={'Acc': Accuracy()})             # 建立模型：将MindSpore Quantum构建的量子机器学习层和MindSpore的算子组合，构成一张更大的机器学习网络


train_loader = NumpySlicesDataset({'features': X_train, 'labels': y_train}, shuffle=False).batch(20) # 通过NumpySlicesDataset创建训练样本的数据集，shuffle=False表示不打乱数据，batch(5)表示训练集每批次样本点有5个
test_loader = NumpySlicesDataset({'features': X_test, 'labels': y_test},shuffle=False).batch(20)                   # 通过NumpySlicesDataset创建测试样本的数据集，batch(5)表示测试集每批次样本点有5个


class StepAcc(ms.Callback):                                                      # 定义一个关于每一步准确率的回调函数
    def __init__(self, model, test_loader):
        self.model = model
        self.test_loader = test_loader
        self.acc = []

    def on_train_step_end(self, run_context):
        
        self.acc.append(self.model.eval(self.test_loader, dataset_sink_mode=False)['Acc'])
        # print(f'ACC = {self.acc[-1]}')


monitor = LossMonitor(20)                                                       # 监控训练中的损失，每16步打印一次损失值

acc = StepAcc(model, test_loader)                                               # 使用建立的模型和测试样本计算预测的准确率

model.train(15, train_loader, callbacks=[monitor, acc], dataset_sink_mode=False)# 将上述建立好的模型训练20次

ValueError: Encoder data requires a two dimension Tensor with second dimension should be 19, but get shape (20, 10)

In [ ]:
plt.plot(acc.acc)
plt.title('Statistics of accuracy', fontsize=20)
plt.xlabel('Steps', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)